In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib
import matplotlib.patches as mpatches

/home/lulu/.pyenv/versions/3.9.7/envs/study_1_venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
spatial_units = gpd.read_file('../data/processed_data/zones_delineation/PC_Delft.gpkg')
spatial_units = spatial_units.drop(columns = ['Postcode','gemeente','area_ratio'])

In [87]:
boundaries = spatial_units.dissolve()

In [4]:
exposure = pd.read_csv('../data/results/exposure/Delft_exposure.csv', usecols= ['id_unit','expos_NW'])

In [5]:
spatial_units = spatial_units.merge(exposure, on = 'id_unit')
average_expos = spatial_units['expos_NW'].mean()
std_expos = spatial_units['expos_NW'].std()

In [6]:
regions = gpd.read_file('../data/results/regions/regions.gpkg')

In [7]:
regions_city = regions.loc[(regions['seg'] == 1) & (regions['city'] == 'Delft')]

In [71]:
spatial_units['exp_norm'] = 0
spatial_units['exp_norm'] = (spatial_units['expos_NW']-average_expos)/std_expos

In [83]:
color_seg = ['#f7d500','#d65800','#a11096']
spatial_units['color'] = color_seg[1]
spatial_units['color'] = spatial_units['color'].mask(spatial_units['exp_norm'] <= -1/3,color_seg[0])
spatial_units['color'] = spatial_units['color'].mask(spatial_units['exp_norm'] > 1/3,color_seg[2])

In [112]:
fig, ax = plt.subplots(1, figsize=(8, 8))
ax.margins(0)
spatial_units.plot(ax=ax,
                   color = spatial_units['color'],
                   edgecolor = spatial_units['color'])

regions_city.plot(ax=ax,color = 'none',hatch='\\\\')

# legend
patch_1 = mpatches.Patch(color=color_seg[0], label='$< \mu - \sigma^2/3$')
patch_2 = mpatches.Patch(color=color_seg[1], label='$[\mu - \sigma^2/3 , \mu + \sigma^2/3]$')
patch_3 = mpatches.Patch(color=color_seg[2], label='$\geq \mu + \sigma^2/3$')
patch_4 = mpatches.Patch(facecolor='white',edgecolor = 'black',hatch='\\\\\\', label='Segregated regions')

ax.legend(handles = [patch_1,patch_2,patch_3,patch_4])
boundaries.plot(ax=ax,color = 'none', edgecolor = 'black')
#df_zones.plot(ax=ax, color  = 'none', edgecolor='black')
ax.axis('off')
ax.add_artist(ScaleBar(dx =1 ,length_fraction=0.3, location = 'lower left'))
#fig.savefig('../report/figures/all_cities/Delft.png')
plt.close()